# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 4

In [2]:
import sys

In [3]:
sys.path.append('..')

In [4]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson
from py_files.fbref_data import fetch_player_data, fetch_team_data
from pathlib import Path

# allow more data columns to be shown than by default
pd.set_option('max_columns',500)

In [5]:
# download player data from fbref.com
playerStats = fetch_player_data()
# save player stats
path = Path(f'../data/fbref/player_stats_week{latest_gameweek}.csv')
playerStats.to_csv(path)
playerStats.head()

General                                                               \
       Rk           Player   Nation Pos            Squad     Age  Born   
0       1       Max Aarons  eng ENG  DF     Norwich City  21-255  2000   
1       2        Che Adams  sct SCO  FW      Southampton  25-065  1996   
2       3  Rayan Aït Nouri   fr FRA  FW           Wolves  20-102  2001   
3       4  Kristoffer Ajer   no NOR  DF        Brentford  23-152  1998   
4       5       Nathan Aké   nl NED  DF  Manchester City  26-210  1995   

  Playing Time                  Performance                              \
            MP Starts  Min  90s         Gls Ast G-PK PK PKatt CrdY CrdR   
0            4      4  360  4.0           0   0    0  0     0    1    0   
1            3      3  250  2.8           0   1    0  0     0    0    0   
2            1      0    7  0.1           0   0    0  0     0    0    0   
3            4      4  340  3.8           0   0    0  0     0    1    0   
4            1      1   90  1.0           0   0    0  0     0    0    0   

  Per 90 Minutes                         Expected                    \
             Gls   Ast   G+A G-PK G+A-PK       xG npxG   xA npxG+xA   
0            0.0  0.00  0.00  0.0   0.00      0.1  0.1  0.4     0.5   
1            0.0  0.36  0.36  0.0   0.36      0.3  0.3  0.9     1.2   
2            0.0  0.00  0.00  0.0   0.00      0.0  0.0  0.0     0.0   
3            0.0  0.00  0.00  0.0   0.00      0.1  0.1  0.3     0.4   
4            0.0  0.00  0.00  0.0   0.00      0.2  0.2  0.0     0.2   

  Per 90 Minutes                            General  
              xG    xA xG+xA  npxG npxG+xA  Matches  
0           0.03  0.10  0.13  0.03    0.13  Matches  
1           0.13  0.32  0.45  0.13    0.45  Matches  
2           0.35  0.00  0.35  0.35    0.35  Matches  
3           0.02  0.08  0.10  0.02    0.10  Matches  
4           0.20  0.00  0.20  0.20    0.20  Matches

In [6]:
# download team data from fbref.com
teamStats = fetch_team_data()
teamStats.set_index('Squad', inplace=True)
# save team stats
#path = Path(f'../data/fbref/team_stats_week{latest_gameweek}.csv')
#teamStats.to_csv(path)
teamStats

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Squad,,,,,,,,,,,,,,,,,,
Manchester Utd,1,4,3,1,0,11,3,8,10,5.8,3.9,1.8,0.46,W D W W,72732,Bruno Fernandes - 4,David de Gea,NaN
Chelsea,2,4,3,1,0,9,1,8,10,5.7,4.8,1.0,0.24,W W D W,38965,Romelu Lukaku - 3,Edouard Mendy,NaN
Liverpool,3,4,3,1,0,9,1,8,10,11.7,4.2,7.5,1.87,W W D W,53296,Mohamed Salah - 3,Alisson,NaN
Everton,4,4,3,1,0,10,4,6,10,7.4,4.4,3.0,0.74,W D W W,38487,"Demarai Gray, Dominic Calvert-Lewin - 3",Jordan Pickford,NaN
Manchester City,5,4,3,0,1,11,1,10,9,11.2,2.3,8.9,2.22,L W W W,51857,Ferrán Torres - 2,Ederson,NaN
Brighton,6,4,3,0,1,5,3,2,9,3.9,4.2,-0.2,-0.06,W W L W,30017,Neal Maupay - 2,Robert Sánchez,NaN
Tottenham,7,4,3,0,1,3,3,0,9,3.8,6.3,-2.5,-0.63,W W W L,57967,Son Heung-min - 2,Hugo Lloris,NaN
West Ham,8,4,2,2,0,10,5,5,8,6.6,3.7,2.9,0.73,W W D D,59826,Michail Antonio - 4,Łukasz Fabiański,NaN
Leicester City,9,4,2,0,2,4,6,-2,6,3.6,8.1,-4.4,-1.10,W L W L,32035,Jamie Vardy - 2,Kasper Schmeichel,NaN


In [7]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,80201,0,0,-1,1,0,1,2.0,2.0,0,Bernd,1.0,False,,None,49,80201.jpg,1.3,Leno,1.7,False,None,a,1,3,4,46713,3562,96960,19998,0.2,0.8,Leno,270,0,0,0,9,0,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,77,14,515,52,488,50,207,15,NaN,,NaN,,NaN,
2,0.0,0.0,115918,0,0,0,0,0,1,0.0,0.0,0,Rúnar Alex,0.0,False,Joined OH Leuven on a season-long loan - Expec...,2021-08-31T22:00:09.069158Z,40,115918.jpg,0.0,Rúnarsson,1.1,False,None,u,1,3,0,19017,0,38729,6366,0.0,0.0,Rúnarsson,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,382,35,373,17,315,14,390,35,NaN,,NaN,,NaN,
3,0.0,0.0,47431,0,0,-2,2,0,3,0.0,0.0,0,Willian,0.0,False,Transferred to Corinthians,2021-08-20T09:30:14.065783Z,63,47431.jpg,0.0,Borges Da Silva,0.1,False,None,u,1,3,0,914,0,18312,892,0.0,0.0,Willian,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,592,241,591,241,589,241,592,241,NaN,,NaN,,NaN,
4,100.0,100.0,54694,0,0,-1,1,0,4,4.7,4.7,9,Pierre-Emerick,3.7,False,,2021-08-13T14:30:14.477664Z,99,54694.jpg,3.7,Aubameyang,2.2,False,None,a,1,3,11,137911,31625,185659,18272,0.4,1.1,Aubameyang,177,1,0,1,4,0,0,0,0,0,0,3,35,40.8,25.2,122.0,18.7,191,21,165,24,30,11,78,18,NaN,,NaN,,1.0,
5,NaN,NaN,58822,0,0,-1,1,0,2,1.3,1.3,1,Cédric,0.3,False,,None,44,58822.jpg,0.3,Soares,0.2,False,None,a,1,3,1,8299,744,13640,1784,0.1,0.2,Cédric,188,0,0,0,7,0,0,0,1,0,0,0,21,30.8,36.1,1.0,6.8,218,87,121,24,293,98,225,72,NaN,,5.0,,NaN,


In [8]:
# list all team names
team_names = ['Arsenal', 'Aston Villa', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton',
        'Leicester City', 'Leeds United', 'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 
        'Norwich City', 'Southampton', 'Tottenham', 'Watford', 'West Ham', 'Wolves']
# assign proper team names for each player
df['team_name'] = [team_names[i] for i in df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [9]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Manchester Utd,1,4,3,1,0,11,3,8,10,5.8,3.9,1.8,0.46,W D W W,72732,Bruno Fernandes - 4,David de Gea,NaN,1.450,0.975,0.377192
Chelsea,2,4,3,1,0,9,1,8,10,5.7,4.8,1.0,0.24,W W D W,38965,Romelu Lukaku - 3,Edouard Mendy,NaN,1.425,1.200,0.301194
Liverpool,3,4,3,1,0,9,1,8,10,11.7,4.2,7.5,1.87,W W D W,53296,Mohamed Salah - 3,Alisson,NaN,2.925,1.050,0.349938
Everton,4,4,3,1,0,10,4,6,10,7.4,4.4,3.0,0.74,W D W W,38487,"Demarai Gray, Dominic Calvert-Lewin - 3",Jordan Pickford,NaN,1.850,1.100,0.332871
Manchester City,5,4,3,0,1,11,1,10,9,11.2,2.3,8.9,2.22,L W W W,51857,Ferrán Torres - 2,Ederson,NaN,2.800,0.575,0.562705
Brighton,6,4,3,0,1,5,3,2,9,3.9,4.2,-0.2,-0.06,W W L W,30017,Neal Maupay - 2,Robert Sánchez,NaN,0.975,1.050,0.349938
Tottenham,7,4,3,0,1,3,3,0,9,3.8,6.3,-2.5,-0.63,W W W L,57967,Son Heung-min - 2,Hugo Lloris,NaN,0.950,1.575,0.207008
West Ham,8,4,2,2,0,10,5,5,8,6.6,3.7,2.9,0.73,W W D D,59826,Michail Antonio - 4,Łukasz Fabiański,NaN,1.650,0.925,0.396531
Leicester City,9,4,2,0,2,4,6,-2,6,3.6,8.1,-4.4,-1.10,W L W L,32035,Jamie Vardy - 2,Kasper Schmeichel,NaN,0.900,2.025,0.131994


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [10]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
#df['games played'] = df['total_points']/df['points_per_game']
#xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
# from last season
xA_proportion = 0.743

In [11]:
if latest_gameweek>1:
    df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
elif latest_gameweek == 1:
    df_previous_week = pd.read_csv('../data/fpl/pre_season.csv', index_col=0)
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,adjusted points,adjusted points per game,assists,assists_week1,assists_week2,assists_week3,bonus,bonus_week1,bonus_week2,bonus_week3,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheet_points,clean_sheets,cleansheet_week1,cleansheet_week2,cleansheet_week3,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,form 10,form 15,form 20,form 5,games played,gameweek,gameweek 10 prediction,gameweek 11 prediction,gameweek 12 prediction,gameweek 2 prediction,gameweek 3 prediction,gameweek 4 prediction,gameweek 5 prediction,gameweek 6 prediction,gameweek 7 prediction,gameweek 8 prediction,gameweek 9 prediction,goals_conceded,goals_scored,goals_week1,goals_week2,goals_week3,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,team_name,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value,value form 10,value form 5,valuePoints metric,value_form,value_next10,value_season,web_name,xA,xA_points,xA_week1,xA_week2,xA_week3,xG,xG+xA,xG+xA per game,xGA_week1,xGA_week2,xGA_week3,xG_points,xG_week1,xG_week2,xG_week3,xPoints,xPoints week 1,xPoints week 2,xPoints week 3,xPoints_next10,yellow_cards,gameweek 13 prediction
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.833801,1.611267,0,0.0,0.0,0.0,0,0.0,0.0,0.0,48,NaN,NaN,0.833801,0,0.0,0.0,0.0,80201,NaN,NaN,0,0,-1,1,0.0,482,50,NaN,NaN,0,1,2.3,0.8,1,Bernd,1.3,NaN,NaN,NaN,NaN,3.0,3,4.296421,4.191449,3.759851,NaN,NaN,4.343405,4.257404,3.977591,3.977591,4.342767,4.332475,9,0,0.0,0.0,0.0,7.9,161,7,False,79.0,41,6,270,NaN,NaN,49,0,0,NaN,0,NaN,80201.jpg,1.3,0,9,Leno,1.8,False,NaN,a,1,3,Arsenal,0.0,453,48,4,10796,3005,52903,4737,0.328830,NaN,NaN,0.727896,0.3,8.537500,0.8,Leno,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.3,2.9,3.8,0.0,0.0,0.0,0.0,1.089483,2.090127,2.220093,1.089483,41.833752,0,4.354798
2,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,100.0,100.0,NaN,0,NaN,NaN,NaN,115918,NaN,NaN,0,0,0,0,0.0,342,16,NaN,NaN,0,1,1.0,-0.5,0,Rúnar Alex,0.0,NaN,NaN,NaN,NaN,NaN,3,3.596501,3.361894,2.595309,NaN,NaN,3.818299,3.732298,3.124741,3.184381,3.253078,3.242787,0,0,NaN,NaN,NaN,0.0,363,33,False,0.0,353,33,0,NaN,2021-08-20T09:30:14.051083Z,40,0,0,NaN,0,NaN,115918.jpg,0.0,0,0,Rúnarsson,1.4,False,NaN,a,1,3,Arsenal,0.0,287,13,0,13327,3042,13224,1050,NaN,NaN,NaN,NaN,0.0,8.428921,0.0,Rúnarsson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.715683,0,3.806396
3,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,75.0,50.0,NaN,0,NaN,NaN,NaN,47431,4.0,NaN,0,0,-2,2,0.0,569,233,2.0,NaN,0,3,0.8,-0.2,0,Willian,0.0,NaN,NaN,NaN,NaN,NaN,3,2.615319,2.361716,1.931880,NaN,NaN,2.912004,2.810753,2.076629,2.197949,2.372007,2.361716,0,0,NaN,NaN,NaN,0.0,570,233,False,0.0,569,232,0,Illness - 75% chance of playing,2021-08-20T09:30:14.065783Z,63,0,0,NaN,0,NaN,47431.jpg,0.0,0,0,Borges Da Silva,0.2,False,NaN,d,1,3,Arsenal,0.0,566,232,0,900,36,14850,900,NaN,NaN,NaN,NaN,0.0,3.887966,0.0,Willian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.494185,0,2.854212
4,2.000000,1.000000,0,NaN,0.0,0.0,0,NaN,0.0,0.0,5,100.0,100.0,0.000000,0,NaN,0.0,0.0,54694,NaN,NaN,0,0,-1,1,2.7,253,35,NaN,NaN,0,4,1.7,0.2,1,Pierre-Emerick,0.7,NaN,NaN,NaN,NaN,2.0,3,1.965817,1.859911,1.385978,NaN,NaN,2.240224,2.138973,1.525185,1.586866,1.870203,1.859911,4,0,NaN,0.0,0.0,2.6,271,34,False,5.0,286,31,87,NaN,2021-08-13T14:30:14.477664Z,99,0,0

In [12]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Tottenham,1,3,3,0,0,3,0,3,9,3.7,3.9,-0.2,-0.05,W W W,57967,Son Heung-min - 2,Hugo Lloris,NaN,1.233333,1.300000,0.272532
West Ham,2,3,2,1,0,10,5,5,7,6.0,2.9,3.1,1.03,W W D,59901,Michail Antonio - 4,Łukasz Fabiański,NaN,2.000000,0.966667,0.380349
Manchester Utd,3,3,2,1,0,7,2,5,7,3.6,3.2,0.3,0.11,W D W,72732,"Mason Greenwood, Bruno Fernandes - 3",David de Gea,NaN,1.200000,1.066667,0.344154
Chelsea,4,3,2,1,0,6,1,5,7,4.6,3.3,1.3,0.45,W W D,38965,"Trevoh Chalobah, Reece James... - 1",Edouard Mendy,NaN,1.533333,1.100000,0.332871
Liverpool,5,3,2,1,0,6,1,5,7,7.6,2.9,4.7,1.56,W W D,52591,"Mohamed Salah, Diogo Jota - 2",Alisson,NaN,2.533333,0.966667,0.380349


In [13]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [14]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
bonus_column = 'bonus_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [15]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, ('Expected','xG')].values[0]  
    penalty_attempts = playerStats.loc[indicator,('Performance','PKatt')].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
                df.loc[ix, bonus_column] = df.loc[ix, 'bonus'] - df_previous_week.loc[ix, 'bonus']
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
                df.loc[ix, bonus_column] = df.loc[ix, 'bonus']
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [16]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [17]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, ('Expected','xA')].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [18]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [19]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        df.loc[ix,'games played'] = playerStats.loc[indicator,('Playing Time','MP')].values[0]
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            df.loc[ix,'games played'] = playerStats.loc[first_name_indicator,('Playing Time','MP')].values[0]
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            if name == 'gudmundsson':
                first_name = 'hann berg '
                exceptional_case_indicator = playerStats[('General','Player')].str.lower().str.contains(first_name)
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'gundogan':
                exceptional_case_indicator = \
                    playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('gundo')
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'fabianski':
                exceptional_case_indicator = \
                    (playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('fabia')) & \
                    (playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('ski'))
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'soucek':
                exceptional_case_indicator = \
                    (playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('tom')) & \
                    playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('sou')
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'fabiosilva':
                exceptional_case_indicator = \
                    playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('biosilva')
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'milivojevic':
                exceptional_case_indicator = (playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('milivoje'))
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'emersonroyal':
                exceptional_case_indicator = (playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('emerson')) & \
                                            (playerStats[('General','Squad')] == 'Tottenham')
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            df.loc[ix,'games played'] = playerStats.loc[full_name_indicator,('Playing Time','MP')].values[0]
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats[('General','Squad')]==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                df.loc[ix,'games played'] = playerStats.loc[indicator & team_indicator,('Playing Time','MP')].values[0]
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

590 is a new index. (xG)
590 is a new index. (xGA)
590 is a new index. (xA)
584 is a new index. (xG)
584 is a new index. (xGA)
584 is a new index. (xA)
592 is a new index. (xG)
592 is a new index. (xGA)
592 is a new index. (xA)
586 is a new index. (xG)
586 is a new index. (xGA)
586 is a new index. (xA)
591 is a new index. (xG)
591 is a new index. (xGA)
591 is a new index. (xA)
587 is a new index. (xG)
587 is a new index. (xGA)
587 is a new index. (xA)
579 is a new index. (xG)
579 is a new index. (xGA)
579 is a new index. (xA)
588 is a new index. (xG)
588 is a new index. (xGA)
588 is a new index. (xA)
582 is a new index. (xG)
582 is a new index. (xGA)
582 is a new index. (xA)
583 is a new index. (xG)
583 is a new index. (xGA)
583 is a new index. (xA)


In [20]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('bonus_week' + str(i)) 
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,team_name,games played,adjusted points,xG,xG_points,clean_sheet_points,xGA_week4,cleansheet_week4,xA,xA_points,xG_week4,goals_week4,xPoints week 4,bonus_week4,xA_week4,assists_week4,adjusted points per game,xPoints week 1,xG_week1,goals_week1,bonus_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1,xPoints week 2,xG_week2,goals_week2,bonus_week2,xGA_week2,cleansheet_week2,xA_week2,assists_week2,xPoints week 3,xG_week3,goals_week3,bonus_week3,xGA_week3,cleansheet_week3,xA_week3,assists_week3
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,80201,0,0,-1,1,0,1,2.0,2.0,0,Bernd,1.0,False,,None,49,80201.jpg,1.3,Leno,1.7,False,None,a,1,3,4,46713,3562,96960,19998,0.2,0.8,Leno,270,0,0,0,9,0,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,77,14,515,52,488,50,207,15,NaN,,NaN,,NaN,,Arsenal,3.0,5.363298,0.0,0.0,1.363298,0.7,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.787766,2.090127,0.0,0.0,0.0,1.3,0.0,0.0,0.0,2.220093,0.0,0.0,0.0,2.9,0.0,0.0,0.0,1.089483,0.0,0.0,0.0,3.8,0.0,0.0,0.0
2,0.0,0.0,115918,0,0,0,0,0,1,0.0,0.0,0,Rúnar Alex,0.0,False,Joined OH Leuven on a season-long loan - Expec...,2021-08-31T22:00:09.069158Z,40,115918.jpg,0.0,Rúnarsson,1.1,False,None,u,1,3,0,19017,0,38729,6366,0.0,0.0,Rúnarsson,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,382,35,373,17,315,14,390,35,NaN,,NaN,,NaN,,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,47431,0,0,-2,2,0,3,0.0,0.0,0,Willian,0.0,False,Transferred to Corinthians,2021-08-20T09:30:14.065783Z,63,47431.jpg,0.0,Borges Da Silva,0.1,False,None,u,1,3,0,914,0,18312,892,0.0,0.0,Willian,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,592,241,591,241,589,241,592,241,NaN,,NaN,,NaN,,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100.0,100.0,54694,0,0,-1,1,0,4,4.7,4.7,9,Pierre-Emerick,3.7,False,,2021-08-13T14:30:14.477664Z,99,54694.jpg,3.7,Aubameyang,2.2,False,None,a,1,3,11,137911,31625,185659,18272,0.4,1.1,Aubameyang,177,1,0,1,4,0,0,0,0,0,0,3,35,40.8,25.2,122.0,18.7,191,21,165,24,30,11,78,18,NaN,,NaN,,1.0,,Arsenal,3.0,12.500000,1.3,5.2,0.000000,0.7,1.0,0.1,0.3,1.3,1.0,10.5,3.0,0.1,0.0,4.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.0,0.0,0.0,2.9,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,3.8,0.0,0.0,0.0
5,NaN,NaN,58822,0,0,-1,1,0,2,1.3,1.3,1,Cédric,0.3,False,,None,44,58822.jpg,0.3,Soares,0.2,False,None,a,1,3,1,8299,744,13640,1784,0.1,0.2,Cédric,188,0,0,0,7,0,0,0,1,0,0,0,21,30.8,36.1,1.0,6.8,218,87,121,24,293,98,225,72,NaN,,5.0,,NaN,,Arsenal,3.0,2.663298,0.0,0.0,1.363298,0.7,0.0,0.1,0.3,0.0,0.0,1.0,0.0,0.0,0.0,0.887766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.520093,0.0,0.0,0.0,2.9,0.0,0.1,0.0,-0.910517,0.0,0.0,0.0,3.8,0.0,0.0,0.0


In [21]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False).head(30)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
579,Ronaldo,1.0,13,13.0,10.200000,10.200000
233,Salah,4.0,38,9.5,35.261751,8.815438
177,Calvert-Lewin,3.0,23,7.7,25.540000,8.513333
237,Alexander-Arnold,4.0,34,8.5,32.741004,8.185251
230,Mané,4.0,24,6.0,30.870751,7.717688
529,Lukaku,3.0,23,7.7,22.600000,7.533333
583,Hwang,1.0,8,8.0,7.200000,7.200000
413,Antonio,4.0,39,9.8,28.004000,7.001000
256,Cancelo,4.0,26,6.5,27.074278,6.768569


In [22]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [23]:
total_xG = playerStats[('Expected','xG')].sum()
total_goals = df['goals_scored'].sum()
print('Total goals: ' + str(total_goals))
print('Total xG: ' + str(total_xG))
print('goals per xG: ' + str(total_goals/total_xG))

Total goals: 105
Total xG: 112.4
goals per xG: 0.9341637010676156
